In [1]:
from os.path import abspath
from pyspark.sql import SparkSession

# set default location for [spark-warehouse]
warehouse_location = abspath('spark-warehouse')

In [2]:
spark = (
    SparkSession
    .builder
    .appName("Delta Lake")
    .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.1.1,io.delta:delta-core_2.12:1.0.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.1")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.sql.debug.maxToStringFields","1000")
    .config("spark.kryoserializer.buffer.max","128m")
    .enableHiveSupport()
    .getOrCreate()
)
spark.sparkContext.setLogLevel("WARN")

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
io.delta#delta-core_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-bff85a66-b1bd-4c21-a36e-ecb42ffd7407;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.1.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.1 in central
	found org.apache.spark#spark-token

In [3]:
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.project.id", "hering-datalake-prod")
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

---

In [4]:
from pyspark.sql.avro.functions import from_avro, to_avro
import pyspark.sql.functions as f
from urllib.request import urlopen
import json
from pyspark.sql.types import StringType 
from dateutil import parser

In [11]:
kafka_bootstrap = '172.31.157.27:9092'
topics = 'bnsvher602.linx_db.dbo.TABELAS_PRECO_FILIAL'

In [ ]:
kafka_bootstrap = '172.17.0.201:9093,172.17.0.202:9093,172.17.0.203:9093'
topics = 'bnsvher602.db_mobile.dbo.VENDA'

In [12]:
df = (
        spark
        .read
        .format('kafka')
        .option('kafka.bootstrap.servers', kafka_bootstrap)
        .option('subscribe', topics)
        .option('startingOffsets', 'earliest')
        .load()
        .withColumn('year', f.date_format(f.col('timestamp'), 'yyyy'))
        .withColumn('month', f.date_format(f.col('timestamp'), 'MM'))
        .withColumn('day', f.date_format(f.col('timestamp'), 'dd'))
        .withColumn('hour', f.date_format(f.col('timestamp'), 'HH'))
    )

In [13]:
df.limit(10).show()

KeyboardInterrupt: 

In [ ]:
df_pd = df.where((f.col('day') > 4)).toPandas()

In [ ]:
df_pd

In [ ]:
df.where((f.col('day') > 3) & (f.col ('month') == 6)).show()

In [ ]:
df.where(f.col('offset')==0).limit(10).toPandas()

In [ ]:
df.limit(10).toPandas()['value'][0]

In [14]:
kafka_bootstrap = 'edh-kafka-bootstrap.ingestion.svc:9092'
topics = 'eih.bnsvher602.linx_db.dbo.TABELAS_PRECO_FILIAL'

In [15]:
df = (
        spark
        .read
        .format('kafka')
        .option('kafka.bootstrap.servers', kafka_bootstrap)
        .option('subscribe', topics)
        .option('startingOffsets', 'earliest')
        .load()
    )

In [16]:
df.toPandas()

22/06/24 01:41:33 WARN ClientUtils: Couldn't resolve server edh-kafka-bootstrap.ingestion.svc:9092 from bootstrap.servers as DNS resolution failed for edh-kafka-bootstrap.ingestion.svc
22/06/24 01:41:33 WARN ClientUtils: Couldn't resolve server edh-kafka-bootstrap.ingestion.svc:9092 from bootstrap.servers as DNS resolution failed for edh-kafka-bootstrap.ingestion.svc


Py4JJavaError: An error occurred while calling o120.collectToPython.
: org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.util.UninterruptibleThreadRunner.runUninterruptibly(UninterruptibleThreadRunner.scala:46)
	at org.apache.spark.sql.kafka010.KafkaOffsetReaderConsumer.fetchTopicPartitions(KafkaOffsetReaderConsumer.scala:131)
	at org.apache.spark.sql.kafka010.KafkaOffsetReaderConsumer.fetchPartitionOffsets(KafkaOffsetReaderConsumer.scala:156)
	at org.apache.spark.sql.kafka010.KafkaOffsetReaderConsumer.getOffsetRangesFromUnresolvedOffsets(KafkaOffsetReaderConsumer.scala:401)
	at org.apache.spark.sql.kafka010.KafkaRelation.buildScan(KafkaRelation.scala:67)
	at org.apache.spark.sql.execution.datasources.DataSourceStrategy$.apply(DataSourceStrategy.scala:347)
	at org.apache.spark.sql.catalyst.planning.QueryPlanner.$anonfun$plan$1(QueryPlanner.scala:63)
	at scala.collection.Itera

22/06/24 01:42:02 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-relation-9501b88e-6256-486d-abdb-8832193cf394-driver-0-1, groupId=spark-kafka-relation-9501b88e-6256-486d-abdb-8832193cf394-driver-0] Connection to node -1 (/172.17.0.201:9093) could not be established. Broker may not be available.
22/06/24 01:42:02 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-relation-9501b88e-6256-486d-abdb-8832193cf394-driver-0-1, groupId=spark-kafka-relation-9501b88e-6256-486d-abdb-8832193cf394-driver-0] Bootstrap broker 172.17.0.201:9093 (id: -1 rack: null) disconnected
22/06/24 01:43:12 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-relation-2c6417d2-da69-45b7-bca6-e59d25c6aa78-driver-0-2, groupId=spark-kafka-relation-2c6417d2-da69-45b7-bca6-e59d25c6aa78-driver-0] Connection to node -1 (/172.17.0.201:9092) could not be established. Broker may not be available.
22/06/24 01:43:12 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-relation-2c6417d2-da69

22/06/24 01:52:08 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-relation-2b75709f-2e62-40e2-beeb-0231f1cdc21e-driver-0-3, groupId=spark-kafka-relation-2b75709f-2e62-40e2-beeb-0231f1cdc21e-driver-0] Connection to node -1 (/172.31.157.27:9092) could not be established. Broker may not be available.
22/06/24 01:52:08 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-relation-2b75709f-2e62-40e2-beeb-0231f1cdc21e-driver-0-3, groupId=spark-kafka-relation-2b75709f-2e62-40e2-beeb-0231f1cdc21e-driver-0] Bootstrap broker 172.31.157.27:9092 (id: -1 rack: null) disconnected
22/06/24 01:52:57 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-relation-9501b88e-6256-486d-abdb-8832193cf394-driver-0-1, groupId=spark-kafka-relation-9501b88e-6256-486d-abdb-8832193cf394-driver-0] Connection to node -1 (/172.17.0.201:9093) could not be established. Broker may not be available.
22/06/24 01:52:57 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-relation-9501b88e-62

22/06/24 02:02:51 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-relation-2c6417d2-da69-45b7-bca6-e59d25c6aa78-driver-0-2, groupId=spark-kafka-relation-2c6417d2-da69-45b7-bca6-e59d25c6aa78-driver-0] Connection to node -1 (/172.17.0.201:9092) could not be established. Broker may not be available.
22/06/24 02:02:51 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-relation-2c6417d2-da69-45b7-bca6-e59d25c6aa78-driver-0-2, groupId=spark-kafka-relation-2c6417d2-da69-45b7-bca6-e59d25c6aa78-driver-0] Bootstrap broker 172.17.0.201:9092 (id: -1 rack: null) disconnected
22/06/24 02:03:03 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-relation-2b75709f-2e62-40e2-beeb-0231f1cdc21e-driver-0-3, groupId=spark-kafka-relation-2b75709f-2e62-40e2-beeb-0231f1cdc21e-driver-0] Connection to node -1 (/172.31.157.27:9092) could not be established. Broker may not be available.
22/06/24 02:03:03 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-relation-2b75709f-2e6

In [ ]:
df.count()

In [ ]:
df_pd = (
    df
    .withColumn('fixedValue', f.col("value").cast("string"))
    .limit(10)
    .toPandas()
)

In [ ]:
df_pd['value'][0]

In [ ]:
df.printSchema()

In [ ]:
df_pd = (
    df
    .withColumn('fixedValue', f.col("value").cast("string"))
    .limit(10)
    .toPandas()
)

In [ ]:
df_pd['value'][0]

In [ ]:
string.split('.')[-1]

In [ ]:
df = (
    spark
    .read
    .format('kafka')
    .option('kafka.bootstrap.servers', kafka_bootstrap)
    .option('subscribe', topics)
    .option('startingOffsets', 'earliest')
    .load()
)

In [ ]:
df.limit(10).toPandas()

In [ ]:
df_pd = (
    df
    .withColumn('fixedValue', f.col("value").cast("string"))
    .limit(10)
    .toPandas()
)

In [ ]:
df_pd['fixedValue'][0]

In [ ]:
df.limit(10).toPandas()

In [ ]:
query1 = (
    df
    .writeStream
    .partitionBy('year', 'month', 'day', 'hour')
    .format('avro')
    .option('checkpointLocation', landing_path_estoque + 'checkpoint')
    .start(landing_path_estoque)
)   

In [ ]:
df.count()

In [ ]:
df.count()

In [ ]:
df_pd1 = df.toPandas()

In [ ]:
df_pd1

In [ ]:
stream_venda = (
    spark
    .readStream
    .format('kafka')
    .option('kafka.bootstrap.servers', kafka_bootstrap)
    .option('subscribe', topics)
    .option('startingOffsets', 'earliest')
    .load()
    .withColumn('year', f.date_format(f.col('timestamp'), 'yyyy'))
    .withColumn('month', f.date_format(f.col('timestamp'), 'MM'))
    .withColumn('day', f.date_format(f.col('timestamp'), 'dd'))
    .withColumn('hour', f.date_format(f.col('timestamp'), 'HH'))
    .where(f.col('topic') == 'bnsvher602.db_mobile.dbo.VENDA')
)

In [ ]:
stream_venda_item = (
    spark
    .readStream
    .format('kafka')
    .option('kafka.bootstrap.servers', kafka_bootstrap)
    .option('subscribe', topics)
    .option('startingOffsets', 'earliest')
    .load()
    .withColumn('year', f.date_format(f.col('timestamp'), 'yyyy'))
    .withColumn('month', f.date_format(f.col('timestamp'), 'MM'))
    .withColumn('day', f.date_format(f.col('timestamp'), 'dd'))
    .withColumn('hour', f.date_format(f.col('timestamp'), 'HH'))
    .where(f.col('topic') == 'bnsvher602.db_mobile.dbo.VENDA_ITEM')
)

In [ ]:
query1 = (
    stream_estoque
    .writeStream
    .partitionBy('year', 'month', 'day', 'hour')
    .format('avro')
    .option('checkpointLocation', landing_path_estoque + 'checkpoint')
    .start(landing_path_estoque)
)   

In [ ]:
query2 = (
    stream_venda
    .writeStream
    .partitionBy('year', 'month', 'day', 'hour')
    .format('avro')
    .option('checkpointLocation', landing_path_venda + 'checkpoint')
    .start(landing_path_venda)
)   

In [ ]:
query3 = (
    stream_venda_item
    .writeStream
    .partitionBy('year', 'month', 'day', 'hour')
    .format('avro')
    .option('checkpointLocation', landing_path_venda_item + 'checkpoint')
    .start(landing_path_venda_item)
)   

In [ ]:
query1.awaitTermination()